In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as smf
import warnings
import pandas as pd
import statsmodels.api as sm

warnings.filterwarnings('ignore')
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('select * from weatherinszeged',con=engine)

# No need for an open connection, 
# because you're only doing a single query
engine.dispose()



df.describe()

,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure
count,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.000000,96453.0,96453.000000
mean,11.932678,10.855029,0.734899,10.810640,187.509232,10.347325,0.0,1003.235956
std,9.551546,10.696847,0.195473,6.913571,107.383428,4.192123,0.0,116.969906
min,-21.822222,-27.716667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,4.688889,2.311111,0.600000,5.828200,116.000000,8.339800,0.0,1011.900000
50%,12.000000,12.000000,0.780000,9.965900,180.000000,10.046400,0.0,1016.450000
75%,18.838889,18.838889,0.890000,14.135800,290.000000,14.812000,0.0,1021.090000
max,39.905556,39.344444,1.000000,63.852600,359.000000,16.100000,0.0,1046.380000


In [6]:
df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [25]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   date                 96453 non-null  datetime64[ns, UTC]
 1   summary              96453 non-null  object             
 2   preciptype           96453 non-null  object             
 3   temperature          96453 non-null  float64            
 4   apparenttemperature  96453 non-null  float64            
 5   humidity             96453 non-null  float64            
 6   windspeed            96453 non-null  float64            
 7   windbearing          96453 non-null  float64            
 8   visibility           96453 non-null  float64            
 9   loudcover            96453 non-null  float64            
 10  pressure             96453 non-null  float64            
 11  dailysummary         96453 non-null  object             
dtypes: datetime64[ns, 

In [16]:
columns_to_skip = ['preciptype', 'summary',"dailysummary"]
for column in columns_to_skip:
    # Extract unique values for the current column
    df[column].fillna('NA', inplace=True)  
    unique_values = df[column].unique()
    # Print the column name and its unique values
    print(f"Unique values for column '{column}':")
    for value in unique_values:
        print(value)
    print()

Unique values for column 'preciptype':
rain
snow
null

Unique values for column 'summary':
Partly Cloudy
Mostly Cloudy
Overcast
Foggy
Breezy and Mostly Cloudy
Clear
Breezy and Partly Cloudy
Breezy and Overcast
Humid and Mostly Cloudy
Humid and Partly Cloudy
Windy and Foggy
Windy and Overcast
Breezy and Foggy
Windy and Partly Cloudy
Breezy
Dry and Partly Cloudy
Windy and Mostly Cloudy
Dangerously Windy and Partly Cloudy
Dry
Windy
Humid and Overcast
Light Rain
Drizzle
Windy and Dry
Dry and Mostly Cloudy
Breezy and Dry
Rain

Unique values for column 'dailysummary':
Partly cloudy throughout the day.
Mostly cloudy throughout the day.
Foggy in the evening.
Foggy overnight and breezy in the morning.
Overcast throughout the day.
Partly cloudy until night.
Mostly cloudy until night.
Foggy starting overnight continuing until morning.
Foggy in the morning.
Partly cloudy until evening.
Partly cloudy starting in the morning.
Mostly cloudy starting overnight continuing until night.
Mostly cloudy unt

In [27]:
# Step 2: Prepare the Data
X = df[['humidity', 'windspeed', 'windbearing', 'pressure']]  # Explanatory variables
y = df['temperature']  # Target variable

# Add constant term for intercept
X = sm.add_constant(X)

# Step 3: Fit the OLS Model
model = sm.OLS(y, X).fit()

# Step 4: Interpret the Results
print(model.summary())




                            OLS Regression Results                            
Dep. Variable:            temperature   R-squared:                       0.421
Model:                            OLS   Adj. R-squared:                  0.421
Method:                 Least Squares   F-statistic:                 1.750e+04
Date:                Tue, 23 Apr 2024   Prob (F-statistic):               0.00
Time:                        21:42:18   Log-Likelihood:            -3.2821e+05
No. Observations:               96453   AIC:                         6.564e+05
Df Residuals:                   96448   BIC:                         6.565e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          37.9264      0.233    162.709      